# Exploratory Data Analysis

In [ ]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
import msoffcrypto
from io import BytesIO
import scipy.stats as stats
import numpy as np
import seaborn as sns
from dotenv import load_dotenv
import os

In [ ]:
# Path to the password-protected Excel file
file_path = "data/FinalDataSet.xlsx"

load_dotenv()
password = os.getenv("DATASET_PASSWORD")

# Decrypt the file
with open(file_path, "rb") as f:
    decrypted_file = BytesIO()
    office_file = msoffcrypto.OfficeFile(f)
    office_file.load_key(password=password)
    office_file.decrypt(decrypted_file)

# Load the decrypted file into pandas
df = pd.read_excel(decrypted_file, engine="openpyxl")
df.head()


## Data Cleaning

There appears to be lots of missing values in this dataset which we will need to appropriately handle. 

In [ ]:
# Look for na values 
df.isna().sum()

## Let's Look at the Core 4 Attributes First

In [ ]:
# Make a copy of the original df, but first let's analyze the first 7 columns only 

df_copy = df.iloc[:, :7].copy()
# Now drop the 35 rows with missing values
df_copy.dropna(inplace=True)
df_copy

## Individual Correlations

In [ ]:
# This data is technically a time series
# Group by uniqueID and then check out correlations?

agg_corrs = df_copy.groupby(['UniqueID'])[['medical', 'clinical_reasoning', 'professionalism', 'collaboration']].corr()
agg_corrs


## Group Correlations

In [ ]:
df_copy[['medical', 'clinical_reasoning', 'professionalism', 'collaboration']].corr()

Initial Results:

- It looks like the medical and clinical_reasoning attributes are correlated, while the professionalism and collaboration attributes are also correlated.

## Is the Data Normally Distributed?

Well the data is discretely distributed, so obviously it can't be truly normal. But it still could be approximately normal.

In [ ]:
# Let's QQ plot the medical column
columns = ['medical', 'clinical_reasoning', 'professionalism', 'collaboration']

for c in columns:
    stats.probplot(df_copy[c], dist="norm", plot=plt)
    plt.title(f"QQ plot for {c}")
    plt.show()

## Kolmogorov-Smirnov Test

H0: The data is normally distributed. 
H1: The data takes some other distribution.

Reject h0 in the case that p < 0.05. 

In [ ]:
columns = ['medical', 'clinical_reasoning', 'professionalism', 'collaboration']
for c in columns:
    stat, p = stats.kstest(df_copy[c], 'norm', args=(np.mean(df_copy[c]), np.std(df_copy[c])))
    print(f"KS Test for {c}: {stat}, {p}")

As expected, the data is not normally distributed. 

## Time Series Visualizations

In [ ]:
# Let's plot student scores over time for each category

df_copy.groupby('UniqueID')[['medical', 'clinical_reasoning', 'professionalism', 'collaboration']]

In [ ]:
df_copy.groupby('UniqueID')["medical"].mean()

In [ ]:
block_order = []

for l in ['A', 'B', 'C', 'D']:
    if l == 'A':
        for i in range(1, 8):
            block_order.append(f"{l}{i}")
    else:
        for i in range(1, 7):
            block_order.append(f"{l}{i}")
    

In [ ]:
def get_student_scores(df, student_id):
    return df[df['UniqueID'] == student_id][['medical', 'clinical_reasoning', 'professionalism', 'collaboration']].values

def get_student_block_order(df, student_id):
    return df[df['UniqueID'] == student_id]['strBlock'].values

def plot_student_scores(df, student_id):
    scores = get_student_scores(df, student_id)
    block_order = get_student_block_order(df, student_id)
    
    # Add jitter to scores
    jitter = np.random.uniform(-0.1, 0.1, scores.shape)
    scores_jittered = scores + jitter

    plt.plot(scores_jittered, marker='o', alpha=0.5)
    plt.xticks(range(len(block_order)), block_order)
    plt.title(f"Scores for student {student_id}")
    plt.legend(['medical', 'clinical_reasoning', 'professionalism', 'collaboration'])
    plt.grid(alpha=0.3)
    plt.show()

plot_student_scores(df_copy, 10)

In [ ]:
def plot_aggregate_scores(df):
    # Compute mean scores for each StrBlock
    summary = df.groupby('strBlock', observed = True)[['medical', 'clinical_reasoning', 'professionalism', 'collaboration']].mean()
    summary.plot(marker='o')
    plt.title('Average Scores Across Students')
    plt.ylabel('Average Score')
    plt.grid(alpha=0.3)
    plt.legend(loc='upper left')
    plt.show()

plot_aggregate_scores(df_copy)


## Looking at Services

Do certain services (classes) perform better during certain blocks?

In [ ]:
# Group by 'service' and 'StrBlock' and calculate the mean medical score

columns = ['medical', 'clinical_reasoning', 'professionalism', 'collaboration']

tracker = []

for c in columns:
    

    grouped = df_copy.groupby(['service', 'strBlock'])[c].mean()

    # Reset index to make the grouped data easier to work with
    grouped_reset = grouped.reset_index()

    # Find the StrBlock with the maximum medical score for each service
    max_blocks = grouped_reset.loc[grouped_reset.groupby('service')[c].idxmax()]

    # Display the result
    tracker.append(max_blocks)


In [ ]:
# merge the dataframes together on 'service'


# Example DataFrames for different categories
max_blocks_medical = tracker[0].rename(columns={'strBlock': 'max_block_medical', 'medical': 'max_avg_medical'})
max_blocks_clinical = tracker[1].rename(columns={'strBlock': 'max_block_clinical', 'medical': 'max_avg_clinical'})
max_blocks_professionalism = tracker[2].rename(columns={'strBlock': 'max_block_professionalism', 'medical': 'max_avg_professionalism'})
max_blocks_collaboration = tracker[3].rename(columns={'strBlock': 'max_block_collaboration', 'medical': 'max_avg_collaboration'})

# Start with one DataFrame and merge the others iteratively
merged_df = max_blocks_medical.merge(
    max_blocks_clinical, on='service', how='outer'
).merge(
    max_blocks_professionalism, on='service', how='outer'
).merge(
    max_blocks_collaboration, on='service', how='outer'
)

# Count the number of blocks shared in each row
block_columns = ['max_block_medical', 'max_block_clinical', 'max_block_professionalism', 'max_block_collaboration']

# Add a new column counting the number of shared blocks
merged_df['shared_blocks_count'] = merged_df[block_columns].apply(lambda row: row.nunique(), axis=1)

# Convert the unique count to a count of shared blocks (total columns - unique values)
merged_df['shared_blocks_count'] = len(block_columns) - merged_df['shared_blocks_count']

merged_df.sort_values('shared_blocks_count', ascending=False)



In [ ]:
service_labels = {
    5600 : 'VTMED 5600: Ambulatory and Production Medicine',
    5601: 'VTMED 5601: Community Practice Service:',
    5602: 'VTMED 5602: Small Animal Medicine',
    5603: 'VTMED 5603: Small Animal Surgery',
    5604: 'VTMED 5604:Large Animal Medicine',
    5605: 'VTMED 5605: Large Animal Surgery',
    5606: 'VTMED 5606: Anesthesia',
    5607: 'VTMED 5607: Dermatology',
    5608: 'VTMED 5608: Opthalmology',
    5609: 'VTMED 5609: Anatomic Pathology',
    5610: 'VTMED 5610: Imaging',
    5611: 'VTMED 5611: Small Animal Emergency and Critical Care',
    5613: 'VTMED 5613: Small Animal Orthopedics',
    6600: 'VTMED 6600: Theriogenology',
    6601: 'VTMED 6601: Cardiology',
    6602: 'VTMED 6602: Lab Animal Medicine',
    6603: 'VTMED 6603: Clinical Wildlife, Exotic, and Zoo Animal Medicine',
    6608: 'VTMED 6608: Clinical Oncology',
    6614: 'VTMED 6614: Large Animal Emergency and Critical Care',
    6616: 'VTMED 6616: Small Animal Dentistry',
    6618: 'VTMED 6618: Clinical Neurology',
    6619: 'VTMED 6619: Clinical Pathology',
    6623: 'VTMED 6623: Clinical Rotation in Shelter Medicine',
    6624: 'VTMED 6624: Primary Care Surgery',
    6627: 'VTMED 6627: Farrier',
    6628: 'VTMED 6628: Clinical Sports Medicine and Rehabilitation',
    6629: 'VTMED 6629: Wildlife Medicine'
}